In [99]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd

In [100]:
picsdir = '/kaggle/input/girls-meowmeow1/static'
df = pd.read_csv("/kaggle/input/girls-meowmeow1/markup.csv")
datalen = len(df)

In [136]:
validation_size = 0.25
autotune = tf.data.experimental.AUTOTUNE
img_size = 299
batch_size = 8
dropout = 0.1
lr = 1e-3
decay_steps = 100
decay_rate = 0.96
epochs = 35

In [137]:
def get_image(filename):
    image = tf.image.decode_jpeg(tf.io.read_file(f"{picsdir}/{filename}"), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype = tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

In [138]:
x = [get_image(filename) for filename in df["Filename"].values]

In [139]:
y = [[int(index == mark) for index in range(10)] for mark in df["Mark"].values]

In [140]:
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).prefetch(buffer_size=autotune)

In [141]:
train_part = 0.75
split_index = int(datalen * train_part // batch_size)
trainset = dataset.take(split_index)
validset = dataset.skip(split_index)

In [142]:
model = tf.keras.models.load_model('../input/keras-applications-models/InceptionV3.h5')
for layer in model.layers[300:]:
    layer.trainable = True

In [143]:
model= tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(img_size, img_size, 3)),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
        model,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout, name='top_dropout'),
        tf.keras.layers.Dense(32, activation='elu'),
        tf.keras.layers.Dense(10,'softmax')
    ]
)

In [144]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', patience=5, restore_best_weights=True)

class Callbackss(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get('loss') < 0.063:
            self.model.stop_training = True
            
call = Callbackss()

In [145]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss='categorical_crossentropy')

In [146]:
history=model.fit(trainset,validation_data=validset, epochs=epochs, verbose=1, callbacks=[early_stop, call], use_multiprocessing=True, workers=-1)

Epoch 1/35
67/67 [==============================] - 17s 144ms/step - loss: 3.2782 - val_loss: 19359.5059
Epoch 2/35
67/67 [==============================] - 8s 125ms/step - loss: 2.4933 - val_loss: 2.6848
Epoch 3/35
67/67 [==============================] - 8s 125ms/step - loss: 2.6231 - val_loss: 762.5870
Epoch 4/35
67/67 [==============================] - 8s 125ms/step - loss: 2.3886 - val_loss: 2.8621
Epoch 5/35
67/67 [==============================] - 8s 125ms/step - loss: 2.3569 - val_loss: 2.5364
Epoch 6/35
67/67 [==============================] - 8s 125ms/step - loss: 2.4073 - val_loss: 29.6818
Epoch 7/35
67/67 [==============================] - 8s 124ms/step - loss: 2.3524 - val_loss: 2.3907
Epoch 8/35
67/67 [==============================] - 8s 124ms/step - loss: 2.3882 - val_loss: 113.0440
Epoch 9/35
67/67 [==============================] - 8s 126ms/step - loss: 2.3846 - val_loss: 77.4685
Epoch 10/35
67/67 [==============================] - 8s 125ms/step - loss: 2.2938 - val_l

In [152]:
model.save('model.h5')

ValueError: Unable to create dataset (name already exists)